In [ ]:
# ==================== Landsat 8 处理模块（增强版）====================

def process_landsat8_enhanced(config):
    """
    增强的Landsat 8数据处理函数
    
    参数:
        config: Config对象，包含所有配置参数
        
    配置选项（可选）:
        config.cloud_threshold: 云量阈值（0-1），默认0.2
        config.compute_vegetation: 是否计算植被指数，默认True
        config.compute_water: 是否计算水体指数，默认True
        config.compute_salinity: 是否计算盐度指数，默认True
        config.compute_soil: 是否计算土壤指数，默认True
        config.compute_thermal: 是否计算热红外指数，默认True
        config.compute_interaction: 是否计算交互指数，默认True
        config.compute_unmixing: 是否计算光谱解混，默认True
        config.compute_texture: 是否计算纹理特征，默认True
        config.compute_phenology: 是否计算物候指标，默认True
    
    返回:
        l8_composite: 处理后的Landsat 8合成影像
        target_projection: 目标投影信息
        
    使用示例:
        # 快速处理（只计算基本指数）
        config.compute_texture = False
        config.compute_unmixing = False
        config.compute_phenology = False
        
        # 盐度专用（只计算盐度相关）
        config.compute_vegetation = False
        config.compute_water = False
        config.compute_salinity = True
        config.compute_soil = True
        config.compute_thermal = True
        config.compute_interaction = True
        config.compute_unmixing = False
        config.compute_texture = False
        config.compute_phenology = False
    """
    print("\n" + "="*60)
    print("Landsat 8 数据处理")
    print("="*60)
    
    # 内部函数定义
    def apply_scale_factors(image):
        """
        应用Landsat 8 Collection 2的缩放因子
        根据官方文档：
        - 光学波段: scale = 0.0000275, offset = -0.2
        - 热红外波段: scale = 0.00341802, offset = 149.0
        """
        # 应用光学波段缩放
        optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
        
        # 应用热红外波段缩放
        thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
        
        # 保留其他波段
        return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)
    
    def cloud_mask_l8(image):
        """
        使用QA_PIXEL波段进行云掩膜
        根据官方文档的位掩码定义：
        - Bit 3: Cloud
        - Bit 4: Cloud Shadow
        - Bit 5: Snow
        """
        qa = image.select('QA_PIXEL')
        
        # 创建云掩膜
        cloud_bit_mask = 1 << 3
        cloud_shadow_bit_mask = 1 << 4
        snow_bit_mask = 1 << 5
        
        # 结合所有掩膜条件
        mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
                .And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
                .And(qa.bitwiseAnd(snow_bit_mask).eq(0)))
        
        # 添加云覆盖百分比作为属性
        cloud_percentage = (qa.bitwiseAnd(cloud_bit_mask).gt(0)
                          .reduceRegion(
                              reducer=ee.Reducer.mean(),
                              geometry=image.geometry(),
                              scale=100,
                              maxPixels=1e9
                          ).get('QA_PIXEL'))
        
        return image.updateMask(mask).set('cloud_percentage', cloud_percentage)
    
    def calculate_vegetation_indices(image):
        """计算植被相关指数"""
        # 获取所有需要的波段
        b2 = image.select('SR_B2')  # Blue (0.452-0.512 μm)
        b3 = image.select('SR_B3')  # Green (0.533-0.590 μm)
        b4 = image.select('SR_B4')  # Red (0.636-0.673 μm)
        b5 = image.select('SR_B5')  # NIR (0.851-0.879 μm)
        b6 = image.select('SR_B6')  # SWIR1 (1.566-1.651 μm)
        b7 = image.select('SR_B7')  # SWIR2 (2.107-2.294 μm)
        
        # NDVI - 归一化植被指数
        ndvi = b5.subtract(b4).divide(b5.add(b4)).rename('NDVI')
        
        # EVI - 增强植被指数
        evi = b5.subtract(b4).divide(
            b5.add(b4.multiply(6)).subtract(b2.multiply(7.5)).add(1)
        ).multiply(2.5).rename('EVI')
        
        # SAVI - 土壤调节植被指数 (L=0.5)
        L = 0.5
        savi = b5.subtract(b4).divide(b5.add(b4).add(L)).multiply(1 + L).rename('SAVI')
        
        # MSAVI2 - 改进的土壤调节植被指数
        msavi2 = (b5.multiply(2).add(1).subtract(
            (b5.multiply(2).add(1)).pow(2).subtract(b5.subtract(b4).multiply(8)).sqrt()
        )).divide(2).rename('MSAVI2')
        
        # GNDVI - 绿色归一化植被指数
        gndvi = b5.subtract(b3).divide(b5.add(b3)).rename('GNDVI')
        
        # NBR - 归一化燃烧率（也可用于土壤监测）
        nbr = b5.subtract(b7).divide(b5.add(b7)).rename('NBR')
        
        # 叶面积指数 (LAI) 的简化估算
        lai = ndvi.multiply(3.618).exp().subtract(0.118).rename('LAI')
        
        return image.addBands([ndvi, evi, savi, msavi2, gndvi, nbr, lai])
    
    def calculate_water_indices(image):
        """计算水体相关指数"""
        b2 = image.select('SR_B2')  # Blue
        b3 = image.select('SR_B3')  # Green
        b4 = image.select('SR_B4')  # Red
        b5 = image.select('SR_B5')  # NIR
        b6 = image.select('SR_B6')  # SWIR1
        b7 = image.select('SR_B7')  # SWIR2
        
        # NDWI - 归一化水体指数 (McFeeters, 1996)
        ndwi = b3.subtract(b5).divide(b3.add(b5).add(0.0001)).rename('NDWI')
        
        # MNDWI - 修正的归一化水体指数 (Xu, 2006)
        mndwi = b3.subtract(b6).divide(b3.add(b6).add(0.0001)).rename('MNDWI')
        
        # WRI - 水体比值指数
        wri = b3.add(b4).divide(b5.add(b6).add(0.0001)).rename('WRI')
        
        # AWEI - 自动水体提取指数
        awei = (b2.add(b3.multiply(2.5)).subtract(b5.add(b6).multiply(1.5))
                .subtract(b7.multiply(0.25))).rename('AWEI')
        
        return image.addBands([ndwi, mndwi, wri, awei])
    
    def calculate_salinity_indices(image):
        """计算盐度相关指数（核心功能）"""
        b1 = image.select('SR_B1')  # Coastal aerosol
        b2 = image.select('SR_B2')  # Blue
        b3 = image.select('SR_B3')  # Green
        b4 = image.select('SR_B4')  # Red
        b5 = image.select('SR_B5')  # NIR
        b6 = image.select('SR_B6')  # SWIR1
        b7 = image.select('SR_B7')  # SWIR2
        
        # 经典盐度指数
        si1 = b2.multiply(b4).sqrt().rename('SI1')
        si2 = b2.pow(2).add(b3.pow(2)).add(b4.pow(2)).sqrt().rename('SI2')
        si3 = b3.pow(2).add(b4.pow(2)).sqrt().rename('SI3')
        
        # Khan et al. (2005) 盐度指数
        si_khan = b4.subtract(b5).divide(b4.add(b5).add(0.0001)).rename('SI_Khan')
        
        # 盐度指数 S1-S6 (Abbas & Khan, 2007)
        s1 = b2.divide(b4.add(0.0001)).rename('S1')
        s2 = b2.subtract(b4).divide(b2.add(b4).add(0.0001)).rename('S2')
        s3 = b3.multiply(b4).divide(b2.add(0.0001)).rename('S3')
        s4 = b2.multiply(b5).divide(b4.add(0.0001)).rename('S4')
        s5 = b2.multiply(b4).divide(b3.add(0.0001)).rename('S5')
        s6 = b4.multiply(b5).divide(b3.add(0.0001)).rename('S6')
        
        # NDSI - 归一化盐度指数
        ndsi = b4.subtract(b5).divide(b4.add(b5).add(0.0001)).rename('NDSI')
        
        # 基于SWIR的盐度指数
        swir_si = b6.subtract(b5).divide(b6.add(b5).add(0.0001)).rename('SWIR_SI')
        
        # 综合盐度指数（结合多个波段）
        csi = (b1.add(b2).add(b4)).divide(b5.add(0.0001)).rename('CSI')
        
        # 盐度亮度指数
        sbi = b2.add(b4).add(b5).rename('SBI')
        
        return image.addBands([
            si1, si2, si3, si_khan, s1, s2, s3, s4, s5, s6,
            ndsi, swir_si, csi, sbi
        ])
    
    def calculate_soil_indices(image):
        """计算土壤相关指数"""
        b2 = image.select('SR_B2')  # Blue
        b3 = image.select('SR_B3')  # Green
        b4 = image.select('SR_B4')  # Red
        b5 = image.select('SR_B5')  # NIR
        b6 = image.select('SR_B6')  # SWIR1
        b7 = image.select('SR_B7')  # SWIR2
        
        # BSI - 裸土指数
        bsi = ((b6.add(b4)).subtract(b5.add(b2))).divide(
            (b6.add(b4)).add(b5.add(b2)).add(0.0001)
        ).rename('BSI')
        
        # BI - 亮度指数
        bi = b2.pow(2).add(b3.pow(2)).add(b4.pow(2)).sqrt().divide(3).rename('BI')
        
        # CI - 结皮指数
        ci = b4.subtract(b2).divide(b4.add(b2).add(0.0001)).rename('CI')
        
        # DBSI - 干旱裸土指数
        # 需要先计算NDVI
        ndvi = b5.subtract(b4).divide(b5.add(b4).add(0.0001))
        dbsi = b6.subtract(b3).divide(b6.add(b3).add(0.0001)).subtract(ndvi).rename('DBSI')
        
        # 土壤颜色指数
        sci = b4.subtract(b3).divide(b4.add(b3).add(0.0001)).rename('SCI')
        
        return image.addBands([bsi, bi, ci, dbsi, sci])
    
    def calculate_thermal_indices(image):
        """计算热红外相关指数"""
        b10 = image.select('ST_B10')  # Thermal band (10.60-11.19 μm)
        
        # 获取影像的统计信息用于标准化
        thermal_stats = b10.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=image.geometry(),
            scale=100,
            maxPixels=1e9
        )
        
        min_temp = ee.Number(thermal_stats.get('ST_B10_min'))
        max_temp = ee.Number(thermal_stats.get('ST_B10_max'))
        
        # 标准化地表温度 (0-1)
        lst_norm = b10.subtract(min_temp).divide(max_temp.subtract(min_temp)).rename('LST_norm')
        
        # 温度条件指数 (TCI)
        tci = max_temp.subtract(b10).divide(max_temp.subtract(min_temp)).rename('TCI')
        
        # 添加温度类别
        temp_class = (b10.lt(295).multiply(1)  # Cold
                     .add(b10.gte(295).And(b10.lt(305)).multiply(2))  # Moderate
                     .add(b10.gte(305).And(b10.lt(315)).multiply(3))  # Warm
                     .add(b10.gte(315).multiply(4))  # Hot
                     .rename('Temp_Class'))
        
        return image.addBands([lst_norm, tci, temp_class])
    
    def calculate_interaction_indices(image):
        """计算波段交互和综合指数"""
        # 获取必要的波段和指数
        ndvi = image.select('NDVI')
        lst = image.select('ST_B10')
        b5 = image.select('SR_B5')  # NIR
        b6 = image.select('SR_B6')  # SWIR1
        
        # TVDI - 温度植被干旱指数（简化版）
        # 需要更复杂的计算，这里使用简化方法
        tvdi = lst.divide(ndvi.add(1)).rename('TVDI_simple')
        
        # 温度-NDVI比值
        temp_ndvi_ratio = lst.divide(ndvi.abs().add(0.1)).rename('Temp_NDVI_ratio')
        
        # 盐度-植被综合指数
        svi = image.select('SI1').multiply(ndvi.multiply(-1).add(1)).rename('SVI')
        
        # 水分胁迫指数
        msi = b6.divide(b5.add(0.0001)).rename('MSI')
        
        # 综合环境胁迫指数
        esi = (lst.unitScale(290, 320)
              .add(image.select('SI1').unitScale(0, 0.5))
              .subtract(ndvi.unitScale(-0.2, 0.8))
              .divide(3)).rename('ESI')
        
        return image.addBands([tvdi, temp_ndvi_ratio, svi, msi, esi])
    
    def add_spectral_unmixing_features(image):
        """添加光谱解混特征"""
        # 定义端元（这里使用简化的端元）
        # 实际应用中应该从影像中提取或使用野外测量数据
        # 这些值是归一化反射率，应根据实际研究区调整
        green_vegetation = [0.05, 0.06, 0.04, 0.20, 0.15, 0.10]  # 健康植被
        bare_soil = [0.14, 0.17, 0.22, 0.24, 0.25, 0.23]       # 裸土
        salt_crust = [0.30, 0.34, 0.38, 0.40, 0.42, 0.41]      # 盐壳
        
        # 选择用于解混的波段
        bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
        
        # 创建端元矩阵
        endmembers = ee.Array([green_vegetation, bare_soil, salt_crust])
        
        # 执行解混
        unmixed = image.select(bands).unmix(endmembers)
        
        # 重命名解混结果
        unmixed = unmixed.rename(['GV_fraction', 'Soil_fraction', 'Salt_fraction'])
        
        # 计算解混误差（可选）
        # 注释掉以提高性能，需要时可启用
        # reconstructed = unmixed.matrixMultiply(endmembers)
        # rmse = image.select(bands).subtract(reconstructed).pow(2).reduce(
        #     ee.Reducer.mean()
        # ).sqrt().rename('Unmix_RMSE')
        
        return image.addBands(unmixed)
    
    def add_texture_features(image):
        """添加纹理特征（基于GLCM）"""
        # 选择用于纹理分析的波段（通常使用NIR或红波段）
        nir = image.select('SR_B5')
        
        # 转换为整数（GLCM需要）
        nir_int = nir.multiply(10000).toInt()
        
        # 计算GLCM纹理
        glcm = nir_int.glcmTexture(size=3)  # 3x3窗口
        
        # 选择关键纹理特征
        contrast = glcm.select('SR_B5_contrast').rename('NIR_contrast')
        correlation = glcm.select('SR_B5_corr').rename('NIR_correlation')
        entropy = glcm.select('SR_B5_ent').rename('NIR_entropy')
        homogeneity = glcm.select('SR_B5_idm').rename('NIR_homogeneity')
        
        return image.addBands([contrast, correlation, entropy, homogeneity])
    
    def add_phenology_metrics(image_collection):
        """添加物候指标（用于时间序列）"""
        # 检查集合大小
        collection_size = image_collection.size()
        
        # 只有当影像数量足够时才计算物候指标
        def compute_phenology():
            # 计算NDVI时间序列的统计指标
            ndvi_max = image_collection.select('NDVI').max().rename('NDVI_max')
            ndvi_min = image_collection.select('NDVI').min().rename('NDVI_min')
            ndvi_mean = image_collection.select('NDVI').mean().rename('NDVI_mean')
            ndvi_std = image_collection.select('NDVI').reduce(
                ee.Reducer.stdDev()
            ).rename('NDVI_std')
            
            # NDVI振幅
            ndvi_amplitude = ndvi_max.subtract(ndvi_min).rename('NDVI_amplitude')
            
            # 变化率（使用线性回归）
            def add_time_band(image):
                return image.addBands(
                    ee.Image(image.date().millis()).divide(1e9).rename('time')
                )
            
            collection_with_time = image_collection.map(add_time_band)
            
            # 线性趋势
            linear_fit = collection_with_time.select(['time', 'NDVI']).reduce(
                ee.Reducer.linearFit()
            )
            ndvi_trend = linear_fit.select('scale').rename('NDVI_trend')
            
            return ee.Image.cat([
                ndvi_max, ndvi_min, ndvi_mean, ndvi_std,
                ndvi_amplitude, ndvi_trend
            ])
        
        # 如果影像太少，返回空值影像
        def empty_phenology():
            return ee.Image.constant(0).rename([
                'NDVI_max', 'NDVI_min', 'NDVI_mean', 'NDVI_std',
                'NDVI_amplitude', 'NDVI_trend'
            ]).mask(ee.Image.constant(0))
        
        # 条件执行
        return ee.Algorithms.If(
            collection_size.gt(3),  # 至少需要4张影像
            compute_phenology(),
            empty_phenology()
        )
    
    # 主处理流程开始
    print(f"时间范围: {config.start_date} 至 {config.end_date}")
    print(f"空间范围: 研究区边界")
    
    # 1. 获取Landsat 8影像集合
    l8_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                     .filterDate(config.start_date, config.end_date)
                     .filterBounds(config.boundary)
                     .map(apply_scale_factors)
                     .map(cloud_mask_l8))
    
    # 过滤云量（云量阈值可配置）
    cloud_threshold = getattr(config, 'cloud_threshold', 0.2)
    l8_filtered = l8_collection.filter(ee.Filter.lt('cloud_percentage', cloud_threshold))
    
    collection_size = l8_collection.size().getInfo()
    filtered_size = l8_filtered.size().getInfo()
    
    print(f"\n数据获取统计:")
    print(f"  原始影像数: {collection_size}")
    print(f"  云量过滤后（<{cloud_threshold*100}%）: {filtered_size}")
    
    if filtered_size == 0:
        print("  警告: 没有满足云量条件的影像，使用所有可用影像")
        l8_filtered = l8_collection
    
    # 2. 计算所有指数
    print("\n计算光谱指数...")
    
    # 获取计算选项（默认全部启用）
    compute_vegetation = getattr(config, 'compute_vegetation', True)
    compute_water = getattr(config, 'compute_water', True)
    compute_salinity = getattr(config, 'compute_salinity', True)
    compute_soil = getattr(config, 'compute_soil', True)
    compute_thermal = getattr(config, 'compute_thermal', True)
    compute_interaction = getattr(config, 'compute_interaction', True)
    compute_unmixing = getattr(config, 'compute_unmixing', True)
    compute_texture = getattr(config, 'compute_texture', True)
    
    # 基础处理
    l8_with_indices = l8_filtered
    
    # 根据配置选择性计算指数
    if compute_vegetation:
        print("  - 计算植被指数...")
        l8_with_indices = l8_with_indices.map(calculate_vegetation_indices)
    
    if compute_water:
        print("  - 计算水体指数...")
        l8_with_indices = l8_with_indices.map(calculate_water_indices)
    
    if compute_salinity:
        print("  - 计算盐度指数...")
        l8_with_indices = l8_with_indices.map(calculate_salinity_indices)
    
    if compute_soil:
        print("  - 计算土壤指数...")
        l8_with_indices = l8_with_indices.map(calculate_soil_indices)
    
    if compute_thermal:
        print("  - 计算热红外指数...")
        l8_with_indices = l8_with_indices.map(calculate_thermal_indices)
    
    if compute_interaction:
        print("  - 计算交互指数...")
        # 交互指数依赖于NDVI，如果没有计算植被指数，需要先计算NDVI
        if not compute_vegetation:
            print("    (注：交互指数需要NDVI，自动计算基础NDVI)")
            def add_basic_ndvi(image):
                b4 = image.select('SR_B4')
                b5 = image.select('SR_B5')
                ndvi = b5.subtract(b4).divide(b5.add(b4).add(0.0001)).rename('NDVI')
                return image.addBands(ndvi)
            l8_with_indices = l8_with_indices.map(add_basic_ndvi)
        l8_with_indices = l8_with_indices.map(calculate_interaction_indices)
    
    if compute_unmixing:
        print("  - 计算光谱解混...")
        l8_with_indices = l8_with_indices.map(add_spectral_unmixing_features)
    
    if compute_texture:
        print("  - 计算纹理特征...")
        l8_with_indices = l8_with_indices.map(add_texture_features)
    
    # 3. 创建时间合成
    print("\n创建时间合成...")
    
    # 计算中值合成
    l8_median = l8_with_indices.median()
    
    # 物候指标计算（根据配置）
    compute_phenology = getattr(config, 'compute_phenology', True)
    
    if compute_phenology and compute_vegetation:  # 物候指标需要NDVI
        print("  - 计算物候指标...")
        phenology_metrics = ee.Image(add_phenology_metrics(l8_with_indices))
        l8_composite = l8_median.addBands(phenology_metrics)
    else:
        l8_composite = l8_median
    
    # 5. 裁剪到研究区
    l8_composite = l8_composite.clip(config.boundary)
    
    # 6. 获取投影信息
    first_image = ee.Image(l8_collection.first())
    target_projection = first_image.select('SR_B2').projection()
    
    # 7. 选择输出波段
    all_bands = l8_composite.bandNames().getInfo()
    
    # 基础波段
    base_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
    
    # 指数波段（按类别组织，根据实际计算情况选择）
    vegetation_bands = ['NDVI', 'EVI', 'SAVI', 'MSAVI2', 'GNDVI', 'NBR', 'LAI'] if compute_vegetation else []
    
    # 如果计算了交互指数但没有计算植被指数，需要包含NDVI
    if compute_interaction and not compute_vegetation and 'NDVI' not in vegetation_bands:
        vegetation_bands = ['NDVI']
    
    water_bands = ['NDWI', 'MNDWI', 'WRI', 'AWEI'] if compute_water else []
    salinity_bands = ['SI1', 'SI2', 'SI3', 'SI_Khan', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6',
                      'NDSI', 'SWIR_SI', 'CSI', 'SBI'] if compute_salinity else []
    soil_bands = ['BSI', 'BI', 'CI', 'DBSI', 'SCI'] if compute_soil else []
    thermal_bands = ['LST_norm', 'TCI', 'Temp_Class'] if compute_thermal else []
    interaction_bands = ['TVDI_simple', 'Temp_NDVI_ratio', 'SVI', 'MSI', 'ESI'] if compute_interaction else []
    unmixing_bands = ['GV_fraction', 'Soil_fraction', 'Salt_fraction'] if compute_unmixing else []
    texture_bands = ['NIR_contrast', 'NIR_correlation', 'NIR_entropy', 'NIR_homogeneity'] if compute_texture else []
    phenology_bands = ['NDVI_max', 'NDVI_min', 'NDVI_mean', 'NDVI_std', 'NDVI_amplitude', 'NDVI_trend'] if (compute_phenology and compute_vegetation) else []
    
    # 合并所有波段
    selected_bands = (base_bands + vegetation_bands + water_bands + salinity_bands +
                     soil_bands + thermal_bands + interaction_bands + unmixing_bands +
                     texture_bands + phenology_bands)
    
    # 过滤实际存在的波段
    selected_bands = [b for b in selected_bands if b in all_bands]
    
    # 打印统计信息
    print(f"\n波段统计:")
    print(f"  基础波段: {len([b for b in base_bands if b in selected_bands])}个")
    print(f"  植被指数: {len([b for b in vegetation_bands if b in selected_bands])}个")
    print(f"  水体指数: {len([b for b in water_bands if b in selected_bands])}个")
    print(f"  盐度指数: {len([b for b in salinity_bands if b in selected_bands])}个")
    print(f"  土壤指数: {len([b for b in soil_bands if b in selected_bands])}个")
    print(f"  热红外指数: {len([b for b in thermal_bands if b in selected_bands])}个")
    print(f"  交互指数: {len([b for b in interaction_bands if b in selected_bands])}个")
    print(f"  解混特征: {len([b for b in unmixing_bands if b in selected_bands])}个")
    print(f"  纹理特征: {len([b for b in texture_bands if b in selected_bands])}个")
    print(f"  物候特征: {len([b for b in phenology_bands if b in selected_bands])}个")
    print(f"  总计: {len(selected_bands)}个特征")
    
    # 选择最终波段
    l8_final = l8_composite.select(selected_bands)
    
    print("\n✓ Landsat 8数据处理完成")
    
    return l8_final, target_projection_with_indices.median()
    
    # 添加物候指标（转换为ee.Image）
    phenology_metrics = ee.Image(add_phenology_metrics(l8_with_indices))
    
    # 4. 最终合成
    l8_composite = l8_median.addBands(phenology_metrics)
    
    # 5. 裁剪到研究区
    l8_composite = l8_composite.clip(config.boundary)
    
    # 6. 获取投影信息
    first_image = ee.Image(l8_collection.first())
    target_projection = first_image.select('SR_B2').projection()
    
    # 7. 选择输出波段
    all_bands = l8_composite.bandNames().getInfo()
    
    # 基础波段
    base_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
    
    # 指数波段（按类别组织）
    vegetation_bands = ['NDVI', 'EVI', 'SAVI', 'MSAVI2', 'GNDVI', 'NBR', 'LAI']
    water_bands = ['NDWI', 'MNDWI', 'WRI', 'AWEI']
    salinity_bands = ['SI1', 'SI2', 'SI3', 'SI_Khan', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6',
                      'NDSI', 'SWIR_SI', 'CSI', 'SBI']
    soil_bands = ['BSI', 'BI', 'CI', 'DBSI', 'SCI']
    thermal_bands = ['LST_norm', 'TCI', 'Temp_Class']
    interaction_bands = ['TVDI_simple', 'Temp_NDVI_ratio', 'SVI', 'MSI', 'ESI']
    unmixing_bands = ['GV_fraction', 'Soil_fraction', 'Salt_fraction']
    texture_bands = ['NIR_contrast', 'NIR_correlation', 'NIR_entropy', 'NIR_homogeneity']
    phenology_bands = ['NDVI_max', 'NDVI_min', 'NDVI_mean', 'NDVI_std', 'NDVI_amplitude', 'NDVI_trend']
    
    # 合并所有波段
    selected_bands = (base_bands + vegetation_bands + water_bands + salinity_bands +
                     soil_bands + thermal_bands + interaction_bands + unmixing_bands +
                     texture_bands + phenology_bands)
    
    # 过滤实际存在的波段
    selected_bands = [b for b in selected_bands if b in all_bands]
    
    # 打印统计信息
    print(f"\n波段统计:")
    print(f"  基础波段: {len([b for b in base_bands if b in selected_bands])}个")
    print(f"  植被指数: {len([b for b in vegetation_bands if b in selected_bands])}个")
    print(f"  水体指数: {len([b for b in water_bands if b in selected_bands])}个")
    print(f"  盐度指数: {len([b for b in salinity_bands if b in selected_bands])}个")
    print(f"  土壤指数: {len([b for b in soil_bands if b in selected_bands])}个")
    print(f"  热红外指数: {len([b for b in thermal_bands if b in selected_bands])}个")
    print(f"  交互指数: {len([b for b in interaction_bands if b in selected_bands])}个")
    print(f"  解混特征: {len([b for b in unmixing_bands if b in selected_bands])}个")
    print(f"  纹理特征: {len([b for b in texture_bands if b in selected_bands])}个")
    print(f"  物候特征: {len([b for b in phenology_bands if b in selected_bands])}个")
    print(f"  总计: {len(selected_bands)}个特征")
    
    # 选择最终波段
    l8_final = l8_composite.select(selected_bands)
    
    print("\n✓ Landsat 8数据处理完成")
    
    return l8_final, target_projection_with_indices.median()
    l8_mean = l8_with_indices.mean()
    l8_p25 = l8_with_indices.reduce(ee.Reducer.percentile([25]))
    l8_p75 = l8_with_indices.reduce(ee.Reducer.percentile([75]))
    
    # 添加物候指标
    phenology_metrics = add_phenology_metrics(l8_with_indices)
    
    # 最佳像素合成（基于NDVI最大值）
    def add_ndvi_band(image):
        return image.addBands(image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi_temp'))
    
    best_pixel = (l8_with_indices
                  .map(add_ndvi_band)
                  .qualityMosaic('ndvi_temp')
                  .select(l8_with_indices.first().bandNames()))
    
    # 4. 选择最终合成方式（默认使用中值合成）
    l8_composite = l8_median.addBands(phenology_metrics)
    
    # 5. 裁剪到研究区
    l8_composite = l8_composite.clip(config.boundary)
    
    # 6. 获取投影信息
    first_image = ee.Image(l8_collection.first())
    target_projection = first_image.select('SR_B2').projection()
    
    # 7. 选择输出波段
    all_bands = l8_composite.bandNames().getInfo()
    
    # 基础波段
    base_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
    
    # 指数波段（按类别组织）
    vegetation_bands = ['NDVI', 'EVI', 'SAVI', 'MSAVI2', 'GNDVI', 'NBR', 'LAI']
    water_bands = ['NDWI', 'MNDWI', 'WRI', 'AWEI']
    salinity_bands = ['SI1', 'SI2', 'SI3', 'SI_Khan', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6',
                      'NDSI', 'SWIR_SI', 'CSI', 'SBI']
    soil_bands = ['BSI', 'BI', 'CI', 'DBSI', 'SCI']
    thermal_bands = ['LST_norm', 'TCI', 'Temp_Class']
    interaction_bands = ['TVDI_simple', 'Temp_NDVI_ratio', 'SVI', 'MSI', 'ESI']
    unmixing_bands = ['GV_fraction', 'Soil_fraction', 'Salt_fraction', 'Unmix_RMSE']
    texture_bands = ['NIR_contrast', 'NIR_correlation', 'NIR_entropy', 'NIR_homogeneity']
    phenology_bands = ['NDVI_max', 'NDVI_min', 'NDVI_mean', 'NDVI_std', 'NDVI_amplitude', 'NDVI_trend']
    
    # 合并所有波段
    selected_bands = (base_bands + vegetation_bands + water_bands + salinity_bands +
                     soil_bands + thermal_bands + interaction_bands + unmixing_bands +
                     texture_bands + phenology_bands)
    
    # 过滤实际存在的波段
    selected_bands = [b for b in selected_bands if b in all_bands]
    
    # 打印统计信息
    print(f"\n波段统计:")
    print(f"  基础波段: {len([b for b in base_bands if b in selected_bands])}个")
    print(f"  植被指数: {len([b for b in vegetation_bands if b in selected_bands])}个")
    print(f"  水体指数: {len([b for b in water_bands if b in selected_bands])}个")
    print(f"  盐度指数: {len([b for b in salinity_bands if b in selected_bands])}个")
    print(f"  土壤指数: {len([b for b in soil_bands if b in selected_bands])}个")
    print(f"  热红外指数: {len([b for b in thermal_bands if b in selected_bands])}个")
    print(f"  交互指数: {len([b for b in interaction_bands if b in selected_bands])}个")
    print(f"  解混特征: {len([b for b in unmixing_bands if b in selected_bands])}个")
    print(f"  纹理特征: {len([b for b in texture_bands if b in selected_bands])}个")
    print(f"  物候特征: {len([b for b in phenology_bands if b in selected_bands])}个")
    print(f"  总计: {len(selected_bands)}个特征")
    
    # 选择最终波段
    l8_final = l8_composite.select(selected_bands)
    
    print("\n✓ Landsat 8数据处理完成")
    
    return l8_final, target_projection

# 可视化函数
def visualize_landsat8(Map, l8_image, config):
    """
    Landsat 8数据可视化
    
    参数:
        Map: geemap地图对象
        l8_image: 处理后的Landsat 8影像
        config: 配置对象
    """
    print("\n添加Landsat 8可视化图层...")
    
    # 真彩色合成
    Map.addLayer(
        l8_image.select(['SR_B4', 'SR_B3', 'SR_B2']),
        {'min': 0, 'max': 0.3},
        'L8 True Color (RGB: 4-3-2)'
    )
    
    # 假彩色合成（植被突出）
    Map.addLayer(
        l8_image.select(['SR_B5', 'SR_B4', 'SR_B3']),
        {'min': 0, 'max': 0.4},
        'L8 False Color (RGB: 5-4-3)'
    )
    
    # SWIR合成（地质/土壤）
    Map.addLayer(
        l8_image.select(['SR_B7', 'SR_B6', 'SR_B4']),
        {'min': 0, 'max': 0.4},
        'L8 SWIR (RGB: 7-6-4)',
        False
    )
    
    # 地表温度
    Map.addLayer(
        l8_image.select('ST_B10'),
        {'min': 290, 'max': 320, 'palette': ['blue', 'white', 'red']},
        'Land Surface Temperature'
    )
    
    # NDVI
    Map.addLayer(
        l8_image.select('NDVI'),
        {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']},
        'NDVI'
    )
    
    # 盐度指数SI1
    Map.addLayer(
        l8_image.select('SI1'),
        {'min': 0, 'max': 0.5, 'palette': ['blue', 'yellow', 'red']},
        'Salinity Index (SI1)',
        False
    )
    
    # 盐度综合指数
    Map.addLayer(
        l8_image.select('CSI'),
        {'min': 0, 'max': 2, 'palette': ['green', 'yellow', 'orange', 'red']},
        'Comprehensive Salinity Index',
        False
    )
    
    print("✓ 可视化图层添加完成")